In [67]:
from time import sleep

from beamngpy import BeamNGpy, Scenario, Vehicle, set_up_simple_logging
from beamngpy.sensors import Mesh, Damage, Camera

In [68]:
# bng = BeamNGpy("localhost", 25252, home='/home/robotixx/Downloads/BeamNG.tech.v0.36.4.0', debug=True)
# bng.open(launch=True)
bng = BeamNGpy("localhost", 25252)
bng.open(launch=True)

In [69]:
vehicle = Vehicle("ego_vehicle", model="rockbouncer", license="DHILO", color="Red")
vehicle.attach_sensor('damage', Damage())
scenario = Scenario("industrial", "broken_mesh_test", description="Mesh Analysis")
scenario.add_vehicle(
        vehicle, pos=(408.36, -336.56, 35.54), rot_quat=(0,0, 0.99, -0.17)
    )
scenario.make(bng)
bng.settings.set_deterministic(60)
bng.scenario.load(scenario)
bng.scenario.start()
bng.step(60)


In [ ]:
vehicle.send_damage([
        ('front_left_suspension', 1.0, (0,0,0)),
    ])

In [ ]:
vehicle.queue_lua_command('beamstate.breakGroup("front_left_suspension")')

In [ ]:
vehicle.queue_lua_command('obj:setTireBurst(2, true)')

In [ ]:
vehicle.queue_lua_command('beamstate.breakGroup("engine")')

In [ ]:
vehicle.get_part_config()

In [ ]:
vehicle.poll_sensors

In [ ]:
for k, v in pairs(beamstate.breakGroup) do print(k) end

In [ ]:
vehicles = bng.get_available_vehicles()
vehicles

In [ ]:
vehicle.queue_lua_command('beamstate:breakGroup("wheel_FL")')

In [ ]:
vehicle.get_center_of_gravity()

In [ ]:
vehicle.recover()

In [ ]:
vehicle.queue_lua_command('obj:breakAllBreakGroups()')

In [ ]:
print(vehicle.is_connected)

In [ ]:
bng.step(80)

In [ ]:
vehicle.queue_lua_command('print("Hello from Lua!")')

In [61]:
camera = Camera(
    "front_cam",
    bng,
    vehicle,
    requested_update_time = 0.1,
    is_using_shared_memory=True,
    pos=(-0.3, 1, 2),
    dir=(0, -1, 0),
    field_of_view_y = 90,
    near_far_planes=(0.1, 1000),
    resolution=(1024, 1024),
    is_streaming=True,
    is_render_annotations=True,
    is_render_instance=True,
    is_render_depth=True,
)

In [63]:
for i in range(41):
    bng.control.step(10)

    raw_readings = camera.stream_raw()

    if i % 10 == 0:
        print("Showing images from the stream...")

        images = camera.stream()

        if camera.is_render_colours:
            images["colour"].show()
        if camera.is_render_annotations:
            images["annotation"].show()
        if camera.is_render_depth:
            images["depth"].show()

Showing images from the stream...
Showing images from the stream...



(eog:14036): EOG-CRITICAL **: 16:08:02.274: eog_image_get_file: assertion 'EOG_IS_IMAGE (img)' failed

(eog:14036): GLib-GIO-CRITICAL **: 16:08:02.274: g_file_equal: assertion 'G_IS_FILE (file1)' failed

(eog:14036): EOG-CRITICAL **: 16:08:02.287: eog_image_get_file: assertion 'EOG_IS_IMAGE (img)' failed

(eog:14036): GLib-GIO-CRITICAL **: 16:08:02.287: g_file_equal: assertion 'G_IS_FILE (file1)' failed


Showing images from the stream...



(eog:14036): EOG-CRITICAL **: 16:08:04.260: eog_image_get_file: assertion 'EOG_IS_IMAGE (img)' failed

(eog:14036): GLib-GIO-CRITICAL **: 16:08:04.260: g_file_equal: assertion 'G_IS_FILE (file1)' failed

(eog:14036): EOG-CRITICAL **: 16:08:04.273: eog_image_get_file: assertion 'EOG_IS_IMAGE (img)' failed

(eog:14036): GLib-GIO-CRITICAL **: 16:08:04.273: g_file_equal: assertion 'G_IS_FILE (file1)' failed


Showing images from the stream...



(eog:14036): EOG-CRITICAL **: 16:08:06.237: eog_image_get_file: assertion 'EOG_IS_IMAGE (img)' failed

(eog:14036): GLib-GIO-CRITICAL **: 16:08:06.237: g_file_equal: assertion 'G_IS_FILE (file1)' failed

(eog:14036): EOG-CRITICAL **: 16:08:06.254: eog_image_get_file: assertion 'EOG_IS_IMAGE (img)' failed

(eog:14036): GLib-GIO-CRITICAL **: 16:08:06.254: g_file_equal: assertion 'G_IS_FILE (file1)' failed


Showing images from the stream...



(eog:14036): EOG-CRITICAL **: 16:08:08.230: eog_image_get_file: assertion 'EOG_IS_IMAGE (img)' failed

(eog:14036): GLib-GIO-CRITICAL **: 16:08:08.230: g_file_equal: assertion 'G_IS_FILE (file1)' failed

(eog:14036): EOG-CRITICAL **: 16:08:08.243: eog_image_get_file: assertion 'EOG_IS_IMAGE (img)' failed

(eog:14036): GLib-GIO-CRITICAL **: 16:08:08.243: g_file_equal: assertion 'G_IS_FILE (file1)' failed


In [72]:
vehicle.queue_lua_command('energyStorage.getStorage("mainTank"):setRemainingRatio(0.1)')

In [73]:
vehicle.queue_lua_command('damageTracker.setDamage("engine", "radiatorLeak", true)')